In [1]:
import numpy as np
from hdf5storage import loadmat
import mne
from mne.minimum_norm import read_inverse_operator, apply_inverse_raw
from pathlib import Path

In [2]:
# mat = loadmat("/mnt/D/University/Fall 2025/BCI/Dataset/Task1_Preprocessed/ZAB/gip_ZAB_SNR6_EEG.mat")
mat = loadmat("/home/g4/Documents/BCI_Project/Dataset/Additional/Task3_Preprocessed/ZAB/gip_ZAB_TSR1_EEG.mat")
EEG = mat['EEG']

In [3]:
data = EEG['data']
sfreq = float(EEG['srate'].item())

In [4]:
# Load pre-computed inverse operator
inv_operator = read_inverse_operator('inverse_operator-inv.fif')

Reading inverse operator decomposition from /home/g4/Documents/BCI_Project/BCI_Project/inverse_operator-inv.fif...
    Reading inverse operator info...
    [done]
    Reading inverse operator decomposition...
    [done]
    105 x 105 full covariance (kind = 1) found.
    Read a total of 1 projection items:
        Average EEG reference (1 x 105)  idle
    Noise covariance matrix read.
    24588 x 24588 diagonal covariance (kind = 2) found.
    Source covariance matrix read.
    24588 x 24588 diagonal covariance (kind = 6) found.
    Orientation priors read.
    24588 x 24588 diagonal covariance (kind = 5) found.
    Depth priors read.
    Did not find the desired covariance matrix (kind = 3)
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Read a total of 1 projection items:
        Average EEG reference (1 x 105)  idle
    Source spaces transformed to the inverse solution coordinate frame


In [5]:
# Create raw object
# info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
info = mne.io.read_info("info.fif")
raw = mne.io.RawArray(data[0], info)
# Set average reference (required)
raw.set_eeg_reference('average', projection=True)

    Read a total of 1 projection items:
        Average EEG reference (1 x 105)  idle
Creating RawArray with float64 data, n_channels=105, n_times=132033
    Range : 0 ... 132032 =      0.000 ...   264.064 secs
Ready.
EEG channel type selected for re-referencing


/tmp/ipykernel_9854/2787503043.py:6: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  raw.set_eeg_reference('average', projection=True)


<RawArray | 105 x 132033 (264.1 s), ~105.9 MiB, data loaded>

In [6]:
# Apply inverse to raw data
stc = apply_inverse_raw(raw, inv_operator, 
                        lambda2=1.0/9.0, 
                        method='dSPM',
                        buffer_size=5000,  # process in chunks
                        start=0, 
                        stop=int(60 * sfreq))  # first 60 seconds

Preparing the inverse operator for use...


    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    Created an SSP operator (subspace dimension = 1)
    Created the whitener using a noise covariance matrix with rank 22 (83 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse to raw...
    Picked 105 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    computing inverse and combining the current components (using 6 segments)...
        segment 1 / 6 done..
        segment 2 / 6 done..
        segment 3 / 6 done..
        segment 4 / 6 done..
        segment 5 / 6 done..
        segment 6 / 6 done..
[done]


In [7]:
stc.data.shape

(8196, 30000)

In [8]:
stc.times.shape

(30000,)

In [9]:
stc.save('source_estimate', overwrite=True)

Writing STC to disk...
[done]
